In [1]:
import dataclasses

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sys; sys.path.append("../../../foggy-statslib")
import foggy_statslib.core as fsc
import foggy_statslib.fin as fsf

# MEAN-VARIANCE-OPTIMAL ALLOCATION

Using the ideas from [my portfolio food pyramid](https://github.com/sparshsah/foggy-demo/blob/main/demo/finance/pflio-food.pdf)
and [1-800-STYLS-4-U](https://github.com/sparshsah/foggy-demo/blob/main/demo/finance/styles4u.ipynb),
we can construct a "model portfolio".
I will run up the big categories from bottom to top of the pyramid,
suggesting a sample vehicle for accessing it and ballparking its ex-ante Sharpe.

Let us agree on the following conventions:
* All PM stats (Sharpe, ER, Vol) are
    * Long-term
    * Annualized
    * Excess-of-cash
* If I leave it unqualified, the stat is gross-of-fee, but net-of-tcost-and-financing. So, it doesn't reflect what your manager is going to charge you for it, but it does reflect prime-brokerage commissions, shorting expenses, etc.
* If I say "net", the stat is net of both fees and tcost-and-financing. So, it reflects management and performance fees paid to the manager in addition to the fees described above.
* Here is an important nuance: When you look at the "expense ratio" of a mutual fund, you are looking at a mix of management fees and tcost-and-financing. Sometimes you'll even see a "gross expense ratio" and a lower "net expense ratio", with the latter reflecting a voluntary reimbursement by the manager. For example, if you see a gross expense ratio of 102bps and a net expense ratio of 99bps, then the manager is currently volunteering to reimburse you at an annualized 3bps (perhaps in this case because they desire some marketing benefit associated keeping an expense ratio less than 100bps). But this number does not compare cleanly from fund to fund, and it varies over time even when the manager has made no change to their policy. So, when you want to translate a gross-of-fee net-of-tcost-and-financing Sharpe to a net-net Sharpe, penalize by the _adjusted_ expense ratio. For a more in-depth explanation, see Morningstar's excellent article ["One Expense Ratio to Rule Them All"](https://www.morningstar.com/funds/one-expense-ratio-rule-them-all).

## Sample vehicles

* Beta: We could use the [AQR Multi-Asset Risk-Parity Mutual Fund (AQRNX)](https://funds.aqr.com/funds/multi-asset/aqr-multi-asset-fund/aqrnx), which targets 10% vol, charging 1.06% adjusted expense ratio.
    * Runner-up: [Bridgewater All-Weather Fund](https://www.bridgewater.com/research-and-insights/the-all-weather-story). Pros: Matches AQR's version in terms of diversification across asset classes and geographies, and All-Weather's track record is actually a smidge _longer_. I also just like the story a bit better. Cons: Not marketed to retail investors. Sad!
* Macro: We could use the [AQR Managed Futures Mutual Fund (AQMNX)](https://funds.aqr.com/funds/alternatives/aqr-managed-futures-strategy-fund/aqmnx), which targets 10% vol, charging 1.51% adjusted expense ratio. The core strategy here is trend-following, plus some other stuff they're coy about in the prospectus but which I can confirm is good.
* [Styles](https://github.com/sparshsah/foggy-demo/blob/main/demo/finance/styles4u.ipynb): We could use the [Vanguard Market-Neutral Equity Mutual Fund (VMNFX)](https://investor.vanguard.com/investment-products/mutual-funds/profile/vmnfx), which targets (I infer) 6% vol, charging 0.20% adjusted expense ratio.
    * Runner-up: QSPNX (which is diversified across asset classes and geographies) or QMNNX (which not only adds more alpha-like signals, but also remains diversified across geographies despite---like VMNFX---being just stocks). Cons: They're both much more expensive in the face of a dirt-cheap 20bps. And really, the deciding factor is that we have too much AQR risk in our model portfolio already.
* Special Situations: We could use the [AQR Diversified Arbitrage Mutual Fund (ADANX)](https://funds.aqr.com/funds/alternatives/aqr-diversified-arbitrage-fund/adanx), which targets a bit less than 5% vol but budgets for a bit heavier-than-usual tails (so let's call it effectively 6% vol), charging 1.50% adjusted expense ratio.
    * Runner-up: [Driehaus Event-Driven Fund (DEVDX)](https://www.driehaus.com/funds/driehaus-event-driven-fund). Pros: Targets slightly higher vol, and is slightly cheaper. Cons: They don't do convertible arbitrage, and on top of that they're just not well-hedged (0.70-correlated to SPX, vs ADANX's 0.45).

In [ ]:
# my stylized ex-ante Sharpes
sharpes = pd.Series({
    "beta": 0.80,
    # start with a base of 0.40, and add +0.10 reward for alphas i like e.g. AQR's GSS Factor Momentum
    "macro": 0.50,
    # start with a base of 0.40, and subtract -0.10 penalty for underdiversification (VMNFX is just stocks and just US)
    "styles": 0.30,
    "spec_sits": 0.40,
})
vols = pd.Series({
    "beta": 0.10,
    "macro": 0.10,
    "styles": 0.06,
    "spec_sits": 0.06,
})
fees = pd.Series({
    "beta": 0.0106,
    "macro": 0.0151,
    "styles": 0.0020,
    "spec_sits": 0.0150,
})

# assume uncorrelated
corr = pd.DataFrame(
    np.identity(len(sharpes)),
    index=sharpes.index,
    columns=sharpes.index,
)

# calculate
sharpes_net = (sharpes*vols - fees) / vols
ers_net = sharpes_net*vols
stats = pd.DataFrame({
    "sharpe_net": sharpes_net.apply(lambda v: f"{v:.2f}"),
    "er_net": ers_net.apply(lambda v: f"{v:.1%}"),
    "vol": vols.apply(lambda v: f"{v:.0%}"),
})
stats

In [ ]:
# notional allocation (i.e. $'s, not risk) in a unit-levered portfolio
w = pd.DataFrame(
    {
        "simple_mvo": fsf._simple_mvo(
            sharpe_vector=sharpes_net,
            vol_vector=vols,
            corr_matrix=corr,
        )[0],
        "thumb_on_scaled": pd.Series(
            {
                "beta": 0.40,
                "macro": 0.25,
                "styles": 0.20,
                "spec_sits": 0.15,
            }
        ),
    }
)
w

In [ ]:
pd.DataFrame({
    env: fsf._get_exante_pm_stats_of_w_as_ser(
        w,
        er_vector=ers_net,
        cov_matrix=fsc.compose_cov_matrix(vol_vector=vols, corr_matrix=corr),
        fmt=True,
    )
    for (env, w) in w.items()
})

# CONSERVATIVE VS AGGRESSIVE RISK-TAKING

In [ ]:
@dataclasses.dataclass(frozen=True)
class History:
    """
    Fields:
        r: pd.DataFrame, The geometric returns.
            Index = sims, Columns = years.
        px: pd.DataFrame, The price index.
        dd: pd.DataFrame, Current geometric drawdown level.
    """
    r: pd.DataFrame = dataclasses.field(default_factory=pd.DataFrame)
    px: pd.DataFrame = dataclasses.field(default_factory=pd.DataFrame)
    dd: pd.DataFrame = dataclasses.field(default_factory=pd.DataFrame)
    
    def __init__(self, r: np.ndarray) -> None:
        r = pd.DataFrame(
            r,
            index=pd.Index(range(r.shape[0]), name="sim"),
            columns=pd.Index(range(r.shape[1]), name="year"),
        )
        r.loc[:, 0] = 0
        r = r.clip(lower=-1)
        # override frozen dataclass by reaching into internals
        object.__setattr__(self, "r", r)
        del r
        px = (1 + self.r).cumprod(axis="columns")
        object.__setattr__(self, "px", px)
        del px
        dd = self.px / self.px.expanding().max() - 1
        object.__setattr__(self, "dd", dd)

In [ ]:
def sim_history(
    er: float = 0.10,
    vol: float = 0.10,
    nyears: int = 50,
    nsims: int = 1_000,
) -> History:
    r = np.random.default_rng(seed=42).normal(
        loc=er, scale=vol, size=(nsims, nyears),
    )
    history = History(r=r)
    return history

# "SPX" -- really 15-17% vol, but left tails bring it to like 20% vol
history_aggressive = sim_history(er=0.10, vol=0.20)
# "diversified"
history_conservative = sim_history(er=0.05, vol=0.05)
avg_px = pd.DataFrame(
    {
        "aggressive": history_aggressive.px.median(),
        "conservative": history_conservative.px.median(),
    },
)

In [ ]:
_, ax = plt.subplots(nrows=2)
# terminals
fsc.plot(
    history_aggressive.px.iloc[:, -1],
    kind="density",
    color="red",
    axvline_locs=[avg_px["aggressive"].iloc[-1]],
    axvline_styles=[":"],
    axvline_colors=["red"],
    # plt
    ax=ax[0],
)
fsc.plot(
    history_conservative.px.iloc[:, -1],
    kind="density",
    color="navy",
    axvline_locs=[avg_px["conservative"].iloc[-1],],
    axvline_styles=[":"],
    axvline_colors=["navy"],
    # plt
    title="DISTRIBUTION OF FINAL WEALTH:",
    ylim_bottom=0,
    logx=True,
    xlim_left=0.001,
    xticklabels_fmt=lambda x, _: f"${x:.2f}" if x < 0.99 else f"${x:.0f}",
    xlabel="",
    ax=ax[0],
)
# paths
fsc.plot(
    history_aggressive.px,
    color="red",
    alpha=0.10,
    # plt
    ax=ax[1],
)
fsc.plot(
    history_conservative.px,
    color="navy",
    alpha=0.10,
    ax=ax[1],
)
fsc.plot(
    avg_px,
    color=["red", "navy"],
    # plt
    title="PATH OF WEALTH BY YEAR:",
    yticklabels_fmt=lambda y, _: f"${y:.0f}",
    ylim_bottom=0.10,
    logy=True,
    xlabel="",
    legend=False,
    figsize=(10, 8),
    ax=ax[1],
)
# show
print()
plt.show()

# stats
qq = [0, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 1]
print("\nQuantiles of final wealth:")
print(
    pd.DataFrame(
        {
            "aggressive": history_aggressive.px.iloc[:, -1].quantile(qq),
            "conservative": history_conservative.px.iloc[:, -1].quantile(qq),
        },
        index=pd.Index(qq, name="quantile"),
    )
    .applymap(lambda x: f"${x:,.2f}" if x < 10 else f"${x:,.0f}")
    .rename(index=lambda pctl: f"{pctl * 100:,.0f}th").T
)
print("\nQuantiles of worst drawdown:")
print(
    pd.DataFrame(
        {
            "aggressive": history_aggressive.dd.min(axis="columns").quantile(qq),
            "conservative": history_conservative.dd.min(axis="columns").quantile(qq),
        },
        index=pd.Index(qq, name="quantile"),
    )
    .applymap(lambda x: f"{x:.0%}")
    .rename(index=lambda pctl: f"{pctl * 100:,.0f}th").T
)
print("\nQuantiles of worst annual return:")
print(
    pd.DataFrame(
        {
            "aggressive": history_aggressive.r.min(axis="columns").quantile(qq),
            "conservative": history_conservative.r.min(axis="columns").quantile(qq),
        },
        index=pd.Index(qq, name="quantile"),
    )
    .applymap(lambda x: f"{x:.0%}")
    .rename(index=lambda pctl: f"{pctl * 100:,.0f}th").T
)